In [3]:
import torch
from torchtext import data
from torchtext import datasets

SEED = 2019

torch.manual_seed(SEED)

TEXT = data.Field(include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [1]:
!pip install torchtext==0.6.0

In [4]:
from torchtext import datasets

train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)

downloading aclImdb_v1.tar.gz


aclImdb_v1.tar.gz: 100%|██████████| 84.1M/84.1M [00:10<00:00, 8.27MB/s]


In [5]:
print(vars(train_data.examples[0])) # vars: return __dict__ attributes of a given object

{'text': ['As', 'with', 'all', 'the', 'other', 'reviewers,', 'this', 'movie', 'has', 'been', 'a', 'constant', 'in', 'my', 'mind', 'after', '30', 'years.', 'I', 'recall', 'going', 'to', 'the', 'library', 'researching', 'all', 'that', 'I', 'could', 'on', 'this', 'story.', 'I', 'even', 'wrote', 'to', 'the', 'PBS', 'station', 'for', 'more', 'information.', 'Despite', 'all', 'this,', 'all', 'I', 'was', 'able', 'to', 'find', 'out', 'was', 'that', 'it', 'was', 'a', 'story', 'printed', 'in', 'a', 'newspaper', 'in', 'the', 'early', 'part', 'of', 'the', '1900s.<br', '/><br', '/>Fastward', 'to', '2002,', 'after', 'years', 'of', 'searching', 'ebay', 'for', 'on', 'a', 'weekly', 'basis', 'and', 'there', 'it', 'was,', 'a', 'VHS', 'copy', 'of', 'the', 'movie.', 'There', 'was', 'one', 'other', 'bidder', 'but', 'I', 'was', 'determined', 'to', 'win', 'this', 'movie.', 'The', 'losing', 'bidder', 'wrote', 'me', 'asking', 'for', 'a', 'copy', 'which', 'I', 'gave', 'her.', 'Despite', 'owning', 'a', 'copy,', '

In [7]:
import random

train_data, valid_data = train_data.split(split_ratio= 0.7, random_state = random.seed(SEED))

In [8]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 12250
Number of validation examples: 5250
Number of testing examples: 25000


In [10]:
MAX_VOCAB_SIZE = 25000

TEXT.build_vocab(train_data,
                 max_size = MAX_VOCAB_SIZE,
                 vectors = "glove.6B.100d")  #, unk_init = torch.Tensor.normal_

LABEL.build_vocab(train_data)

In [11]:
print(TEXT.vocab.itos[:9])

['<unk>', '<pad>', 'the', 'a', 'and', 'of', 'to', 'is', 'in']


In [12]:
print(TEXT.vocab.vectors[:3])

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  

In [13]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 25002
Unique tokens in LABEL vocabulary: 2


In [14]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device)

In [16]:
import torch.nn as nn

class MyRNN(nn.Module):
    def __init__(self):

        super().__init__()

        vocab_size = 25002
        embedding_dim = 100
        hidden_dim = 200
        output_dim = 1
        n_layers = 2
        bidirectional = True
        dropout = 0.5
        pad_idx = TEXT.vocab.stoi[TEXT.pad_token]

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)

        self.lstm = nn.LSTM(embedding_dim,
                           hidden_dim,
                           num_layers=n_layers,
                           bidirectional=bidirectional,
                           dropout=dropout)

        self.fc = nn.Linear(hidden_dim * 2, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):

        #text = [sent len, batch size]

        embedded = self.embedding(text)

        #embedded = [sent len, batch size, emb dim]

        output, (hidden, cell) = self.lstm(embedded)

        #concat the final forward (hidden[2,:,:]) and backward (hidden[3,:,:]) hidden layers
        #and apply dropout

        hidden = self.dropout(torch.cat((hidden[2,:,:], hidden[3,:,:]), dim = 1))

        #hidden = [batch size, hid dim * 2]

        return self.fc(hidden)

In [17]:
model = MyRNN()
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')
print(MyRNN())
[(n, p.numel()) for n, p in MyRNN().named_parameters()]

The model has 3,947,001 trainable parameters
MyRNN(
  (embedding): Embedding(25002, 100, padding_idx=1)
  (lstm): LSTM(100, 200, num_layers=2, dropout=0.5, bidirectional=True)
  (fc): Linear(in_features=400, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


[('embedding.weight', 2500200),
 ('lstm.weight_ih_l0', 80000),
 ('lstm.weight_hh_l0', 160000),
 ('lstm.bias_ih_l0', 800),
 ('lstm.bias_hh_l0', 800),
 ('lstm.weight_ih_l0_reverse', 80000),
 ('lstm.weight_hh_l0_reverse', 160000),
 ('lstm.bias_ih_l0_reverse', 800),
 ('lstm.bias_hh_l0_reverse', 800),
 ('lstm.weight_ih_l1', 320000),
 ('lstm.weight_hh_l1', 160000),
 ('lstm.bias_ih_l1', 800),
 ('lstm.bias_hh_l1', 800),
 ('lstm.weight_ih_l1_reverse', 320000),
 ('lstm.weight_hh_l1_reverse', 160000),
 ('lstm.bias_ih_l1_reverse', 800),
 ('lstm.bias_hh_l1_reverse', 800),
 ('fc.weight', 400),
 ('fc.bias', 1)]

In [18]:
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

torch.Size([25002, 100])


In [19]:
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.2159,  0.0606, -0.6899,  ..., -0.4062,  0.2097,  0.4097],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.2357,  0.0803,  0.2598,  ...,  0.0144,  0.0844,  0.6444]])

In [21]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

EMBEDDING_DIM = 100
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.2159,  0.0606, -0.6899,  ..., -0.4062,  0.2097,  0.4097],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.2357,  0.0803,  0.2598,  ...,  0.0144,  0.0844,  0.6444]])


In [22]:
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

In [24]:
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [26]:
def binary_accuracy(preds, y):

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division
    acc = correct.sum() / len(correct)
    return acc

In [31]:
def train(model, iterator, optimizer, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:

        optimizer.zero_grad()

        text, text_lengths = batch.text
        text_lengths = torch.as_tensor(text_lengths, dtype=torch.int64, device='cpu')
        predictions = model(text, text_lengths).squeeze(1)

        loss = criterion(predictions, batch.label)

        acc = binary_accuracy(predictions, batch.label)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [32]:
def evaluate(model, iterator, criterion):

    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for batch in iterator:

            text, text_lengths = batch.text
            text_lengths = torch.as_tensor(text_lengths, dtype=torch.int64, device='cpu')
            predictions = model(text, text_lengths).squeeze(1)

            loss = criterion(predictions, batch.label)

            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [33]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [35]:
N_EPOCHS = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'best-model.pt')

    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 16s
	Train Loss: 0.106 | Train Acc: 96.39%
	 Val. Loss: 0.491 |  Val. Acc: 81.31%
Epoch: 02 | Epoch Time: 0m 16s
	Train Loss: 0.061 | Train Acc: 98.01%
	 Val. Loss: 0.435 |  Val. Acc: 86.33%
Epoch: 03 | Epoch Time: 0m 16s
	Train Loss: 0.024 | Train Acc: 99.37%
	 Val. Loss: 0.580 |  Val. Acc: 86.26%
Epoch: 04 | Epoch Time: 0m 15s
	Train Loss: 0.029 | Train Acc: 99.06%
	 Val. Loss: 0.683 |  Val. Acc: 85.43%
Epoch: 05 | Epoch Time: 0m 15s
	Train Loss: 0.012 | Train Acc: 99.71%
	 Val. Loss: 0.696 |  Val. Acc: 85.86%


In [36]:
model.load_state_dict(torch.load('best-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.522 | Test Acc: 84.24%


In [50]:
import spacy
import string
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok for tok in sentence.split(' ')]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [51]:
predict_sentiment(model, "This film is terrible")

0.002092281822115183

In [52]:
predict_sentiment(model, "This film is great")

0.9994105100631714

In [53]:
predict_sentiment(model, "This film is terribly great")

0.036132343113422394